In [0]:
spark.table("trip_data.trip_source.silver").count()

2870071

In [0]:
from pyspark.sql.functions import col, sum

df = spark.table("trip_data.trip_source.silver")

df.select(
    sum(col("passenger_count").isNull().cast("int")).alias("null_passenger"),
    sum(col("trip_distance").isNull().cast("int")).alias("null_distance"),
    sum(col("fare_amount").isNull().cast("int")).alias("null_fare"),
    sum(col("payment_type").isNull().cast("int")).alias("null_payment")
).show()


+--------------+-------------+---------+------------+
|null_passenger|null_distance|null_fare|null_payment|
+--------------+-------------+---------+------------+
|             0|            0|        0|           0|
+--------------+-------------+---------+------------+



In [0]:
df.select(
    "passenger_count", "trip_distance", "fare_amount"
).describe().show()


df.agg(
    {"passenger_count": "max", "trip_distance": "max", "fare_amount": "max"}
).show()


+-------+------------------+------------------+------------------+
|summary|   passenger_count|     trip_distance|       fare_amount|
+-------+------------------+------------------+------------------+
|  count|           2870071|           2870071|           2870071|
|   mean|1.3366801727204658|3.2933792195379232|18.490033351087764|
| stddev|0.8266985377784943| 4.376772339606363|17.115718389408485|
|    min|               1.0|              0.01|               0.0|
|    max|               9.0|              99.0|             820.0|
+-------+------------------+------------------+------------------+

+--------------------+------------------+----------------+
|max(passenger_count)|max(trip_distance)|max(fare_amount)|
+--------------------+------------------+----------------+
|                 9.0|              99.0|           820.0|
+--------------------+------------------+----------------+



In [0]:
# Passenger count >= 1
df.filter(col("passenger_count") < 1).count()  # should be 0

# No negative fares
df.filter(col("fare_amount") < 0).count()  # should be 0

# Trip distance > 0
df.filter(col("trip_distance") <= 0).count()  # should be 0

# Invalid timestamps
from pyspark.sql.functions import year
df.filter((year(col("tpep_pickup_datetime")) < 2020) | (year(col("tpep_pickup_datetime")) > 2025)).count()  # should be 0


0

In [0]:
spark.table("trip_data.trip_source.silver").columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [0]:
%sql
SELECT DISTINCT DOLocationID FROM trip_data.trip_source.silver;

DOLocationID
48
262
107
137
90
33
211
239
43
236
